In [ ]:
# Authenticate if we haven't already

from azureml.core.authentication import InteractiveLoginAuthentication

credentials = InteractiveLoginAuthentication()

In [ ]:
# Load the workspace info

from azureml.core import Workspace

ws = Workspace.from_config(auth=credentials)

In [ ]:
# Create a new experiment if we haven't already

from azureml.core import Experiment
exp = Experiment(workspace=ws, name='AIBootcamp')

In [ ]:
# Configure the experiment to run in the cloud for simplicity reasons

from azureml.core.compute import ComputeTarget, AmlCompute

compute = ComputeTarget(workspace=ws, name='Morty')
compute.wait_for_completion(show_output=True)

In [ ]:
# Initialize the data store

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
# Upload our dataset to the cloud (don't run this every time though)

ds.upload(src_dir='./data', target_path='data', overwrite=False, show_progress=True)


In [ ]:
ds.path('data').as_mount()

In [ ]:
from azureml.train.estimator import Estimator

args = {
    '--data-folder': ds.path('data').as_mount()
}

est = Estimator(source_directory='.',
                script_params=args,
                compute_target=compute,
                entry_script='train_withuploadeddata.py',
                conda_packages=['pandas', 'scikit-learn'])

In [ ]:
run = exp.submit(config=est)

run.wait_for_completion(show_output=True)

In [ ]:
# and then display the metrics logged during the run

run.get_metrics()